# Intro

Most memory objects assume a single input. 
<br>In this notebook, we go over how to add memory 
<br>to a chain that has multiple inputs. 
<br>
<br>As an example of such a chain, we will add 
<br>memory to a question/answering chain. 
<br>
<br>This chain takes as inputs both related docs and a user question.

In [3]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [4]:
# loads file while chunking it into pieces of 1,000 (token or chars?) each
with open('state_of_the_union.txt', encoding='utf-8') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [5]:
print(text_splitter,"\n")
print(texts[0], len(texts[0]),'\n\n', '+'*50, '\n\n', texts[1], len(texts[1]))


Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 939 

 ++++++++++++++++++++++++++++++++++++++++++++++++++ 


In [6]:
# create an embedding for each text
docsearch = Chroma.from_texts(
    texts, 
    embeddings, 
    metadatas=[{"source": i} for i in range(len(texts))])

### Running Chroma using direct local API.
<br>Using DuckDB in-memory for database. 
<br>Data will be transient.

In [41]:
query = "What did the president say about Justice Breyer?"
docs = docsearch.similarity_search(query)
print('Number of documents that got returned:', len(docs), '\n')
print(docs[0], '\n\n\n\n', docs[1])

Number of documents that got returned: 4 

page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.' metadata={'source': 31} 



 page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. An

In [42]:
# define template to accept user prompt, instruct LLM, and keep history

template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document 
and a question, create a final answer. 

Directly cite the text and do not rephrase any wording.

Please do not include any text aside from the citation.
{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], 
    template=template
)

In [43]:
for ii in prompt:
    print(ii)

('input_variables', ['chat_history', 'human_input', 'context'])
('output_parser', None)
('partial_variables', {})
('template', 'You are a chatbot having a conversation with a human.\n\nGiven the following extracted parts of a long document \nand a question, create a final answer. \n\nDirectly cite the text and do not rephrase any wording.\n\nPlease do not include any text aside from the citation.\n{context}\n\n{chat_history}\nHuman: {human_input}\nChatbot:')
('template_format', 'f-string')
('validate_template', True)


In [44]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    OpenAI(temperature=0), 
    chain_type="stuff", 
    memory=memory, prompt=prompt)

In [47]:
def myquery(query):
    return chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

In [48]:
myquery("What did the president say about Justice Breyer")

{'output_text': ' The President said, "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."'}

In [23]:
myquery("How was the tone of Biden's speech?")

{'output_text': " The tone of President Biden's speech was one of appreciation and respect for Justice Breyer's service to the country, as well as a call to action for the Senate to pass the Freedom to Vote Act, the John Lewis Voting Rights Act, and the Disclose Act."}

In [49]:
myquery("Summarize Biden's speech in three points")

{'output_text': ' President Biden called on the Senate to pass the Freedom to Vote Act, the John Lewis Voting Rights Act, and the Disclose Act. He honored Justice Stephen Breyer for his service and nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court. Lastly, he discussed his plans to secure the border and fix the immigration system.'}

In [25]:
query = "Hey what did I just ask you?"
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': " You asked me to summarize President Biden's speech in three points."}

In [50]:
myquery(" How does Joe Biden define quantum mechanics in this speech?")

{'output_text': ' This speech does not contain any information about quantum mechanics.'}

In [51]:
ss = chain.memory.buffer
print(ss.replace("Human:","\nHuman:"))


Human: What did the president say about Justice Breyer
AI:  The President said, "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."

Human: What did the president say about Justice Breyer
AI:  The President said, "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."

Human: Summarize Biden's speech in three points
AI:  President Biden called on the Senate to pass the Freedom to Vote Act, the John Lewis Voting Rights Act, and the Disclose Act. He honored Justice Stephen Breyer for his service and nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court. L

# Creating Embeddings

In [52]:
import openai
response = openai.Embedding.create(
    input="House house house",
    model='text-embedding-ada-002')

print(response)

{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.015424578450620174,
        0.01622888818383217,
        -0.016721852123737335,
        -0.006862575188279152,
        -0.004696128889918327,
        0.006933925207704306,
        0.021353116258978844,
        -0.0067198751494288445,
        0.005837729200720787,
        -0.01300516352057457,
        0.012641927227377892,
        0.005526383873075247,
        0.00974900834262371,
        -0.0013297050027176738,
        -0.00893821194767952,
        -0.0015145664801821113,
        0.02524493634700775,
        -0.01603429578244686,
        0.025011427700519562,
        -0.023753073066473007,
        -0.023299027234315872,
        -0.002344821346923709,
        -0.008821457624435425,
        0.002339956583455205,
        -0.0019086131360381842,
        -0.0032172375358641148,
        0.010968444868922234,
        -0.011526272632181644,
        0.004212894476950169,
      

In [78]:
embedding_array = response["data"][0]["embedding"]
embedding_length = len(embedding_array)
print(embedding_length)

1536
